<a href="https://colab.research.google.com/github/LediDi060/colab/blob/main/%D0%A1%D1%82%D1%80%D0%B5%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0_%D0%94%D0%B8%D0%B0%D0%BD%D0%B0_%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%E2%84%962.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Этап 1. Подготовка окружения**

Получите доступ к тестовому API антивирусной системы (VirusTotal).
Важно: можно использовать публичный API с бесплатными ключами (например, VirusTotal API) или имитировать ответ через mock-сервер.

**Этап 2. Авторизация и запрос**

• Напишите Python-скрипт, который выполняет авторизацию по API-ключу или другому методу. Если используется публичное API, настройте переменную окружения или передайте ключ в заголовках.

• Выполните запрос, возвращающий JSON-ответ. Например, получение статуса сканирования файла или правил фильтрации трафика.

**Этап 3. Вывод результатов**

• Сохраните или выведите полученный JSON-ответ в консоль.

• Добавьте описание в виде комментариев в скрипте, как его запустить и какие переменные предварительно задать, например, API-ключ.


In [6]:
"""
Лабораторная работа 2: Работа с API системы защиты информации (VirusTotal)
Студентка: Стрельникова Диана Александровна

Этапы выполнения:
1. Подготовка окружения - получение доступа к тестовому API
2. Авторизация и выполнение запроса к API
3. Вывод результатов в консоль

Критерии приемки:
✓ Установлены библиотеки requests, json
✓ Выполнена авторизация при обращении к API
✓ Скрипт получил и корректно вывел JSON-ответ
✓ Содержит описание запуска и настройки переменных
✓ Результат соответствует требованиям задания
"""

# ============================================
# Этап 1: ПОДГОТОВКА ОКРУЖЕНИЯ
# ============================================
print("ЭТАП 1: Подготовка окружения")

import requests
import json
import subprocess
import sys

# Публичный демонстрационный API-ключ VirusTotal
API_KEY = "fbbbaa8a0abfac2d48ed428492de91f1995a0c464660d44ae0c88309d9f19084"

# SHA-256 хеш тестового файла (EICAR тестовый антивирусный файл)
FILE_HASH = "275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f"

print(f"\nКонфигурация:")
print(f"• API ключ: {API_KEY[:15]}... (первые 15 символов)")
print(f"• Хеш файла: {FILE_HASH}")
print(f"• API сервер: VirusTotal")

# ============================================
# Этап 2: АВТОРИЗАЦИЯ И ЗАПРОС
# ============================================
print("=" * 60)
print("\n \nЭТАП 2: Авторизация и запрос")

URL = f"https://www.virustotal.com/api/v3/files/{FILE_HASH}"

headers = {
    "x-apikey": API_KEY,  # Авторизация по API-ключу
    "Accept": "application/json"  # Ожидаем JSON-ответ
}

print("Настройка запроса:")
print(f"URL: {URL}")
print(f"Метод: GET")
print(f"Заголовки: x-apikey (авторизация), Accept (json)")

# Выполнение запроса к API
print("\nВыполнение запроса к VirusTotal API...")
try:
    response = requests.get(URL, headers=headers, timeout=30)
    print(f"Запрос выполнен. Статус: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Ошибка запроса: {e}")
    exit(1)

# Проверка ответа от сервера
print(f"\nАнализ ответа сервера:")
print(f"Статус код: {response.status_code}")

if response.status_code == 200:
    print("Результат: Успешный запрос")
elif response.status_code == 401:
    print("Результат: Ошибка авторизации (неверный API-ключ)")
elif response.status_code == 404:
    print("Результат: Файл не найден в базе")
elif response.status_code == 429:
    print("Результат: Превышен лимит запросов (4 в минуту для публичного API)")

# ============================================
# Этап 3: ВЫВОД РЕЗУЛЬТАТОВ
# ============================================
print("=" * 60)
print("\n \nЭТАП 3: Вывод результатов")

if response.status_code == 200:
    try:
        data = response.json()

        print("JSON-ответ успешно получен")
        print(f"Тип данных: {type(data)}")
        print(f"Структура ответа содержит ключи: {list(data.keys())}")

        print("\n \n Полный JSON-ответ:")
        print(json.dumps(data, indent=2, ensure_ascii=False))

        if "data" in data:
            file_data = data["data"]

            if "attributes" in file_data:
                attrs = file_data["attributes"]

                if "last_analysis_stats" in attrs:
                    stats = attrs["last_analysis_stats"]
                    print(f"\n \n Статистика:")
                    print(f"Без угроз: {stats.get('harmless', 0)}")
                    print(f"Подозрительные: {stats.get('suspicious', 0)}")
                    print(f"Злонамеренные: {stats.get('malicious', 0)}")
                    print(f"Не обнаружено: {stats.get('undetected', 0)}")

                    # Определение статуса файла
                    malicious = stats.get('malicious', 0)
                    suspicious = stats.get('suspicious', 0)

                print(f"\n Полный отчет:")
                print(f"Ссылка на отчет: https://www.virustotal.com/gui/file/{FILE_HASH}")

    except json.JSONDecodeError as e:
        print(f"Ошибка парсинга JSON: {e}")
        print(f"Ответ сервера: {response.text[:200]}...")
    except KeyError as e:
        print(f"Ошибка доступа к ключу в JSON: {e}")
        print(f"Структура ответа: {data.keys()}")

else:
    print(f"Не удалось получить данные. Ответ сервера:")
    print(response.text)
    print("-" * 60)

# ============================================
# ИНСТРУКЦИЯ ПО ЗАПУСКУ
# ============================================
print("=" * 60)
print("\n \n \n Инструкция")

print("""
Алгоритм запуска:

1. Установка библиотек:
   Если библиотеки не установлены, выполните:
   pip install requests

2. Настройка переменных:
   В коде скрипта настройте:
   • API_KEY - ваш ключ VirusTotal
   • FILE_HASH - SHA-256 хеш проверяемого файла

3. Запуск:
   python virus_total_api.py

4. Тестовые данные:
   • API ключ: публичный демонстрационный
   • Хеш файла: EICAR тестовый файл
""")
# ============================================
# ЗАВЕРШЕНИЕ РАБОТЫ
# ============================================
print("=" * 60)
print(" \n \n Результат выполнения лабораторной работы №2")

if response.status_code == 200:
    print("\n Выполнены этапы:")
    print("1. Подготовка окружения - библиотеки установлены")
    print("2. Авторизация и запрос - API ключ валиден")
    print("3. Вывод результатов - JSON получен и обработан")
else:
    print(f"Причина: Ошибка запроса (статус {response.status_code})")

ЭТАП 1: Подготовка окружения

Конфигурация:
• API ключ: fbbbaa8a0abfac2... (первые 15 символов)
• Хеш файла: 275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f
• API сервер: VirusTotal

 
 ЭТАП 2: Авторизация и запрос
Настройка запроса:
URL: https://www.virustotal.com/api/v3/files/275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f
Метод: GET
Заголовки: x-apikey (авторизация), Accept (json)

 Выполнение запроса к VirusTotal API...
Запрос выполнен. Статус: 200

Анализ ответа сервера:
Статус код: 200
Результат: Успешный запрос

 
 ЭТАП 3: Вывод результатов
JSON-ответ успешно получен
Тип данных: <class 'dict'>
Структура ответа содержит ключи: ['data']

 
 Полный JSON-ответ:
{
  "data": {
    "id": "275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f",
    "type": "file",
    "links": {
      "self": "https://www.virustotal.com/api/v3/files/275a021bbfb6489e54d471899f7db9d1663fc695ec2fe2a2c4538aabf651fd0f"
    },
    "attributes": {
      "saferpickle